# 1. Workflow for building and deploying interactive dashboards

**Let's say you want to make it easy to explore some dataset. That is, you want to:** 

* Make a visualization of the data
* Maybe add some custom widgets to see the effects of some variables
* Then deploy the result as a web app.

**You can definitely do that in Python, but you would expect to:**
* Spend days of effort to get some initial prototype working in a Jupyter notebook
* Work hard to tame the resulting opaque mishmash of domain-specific, widget, and plotting code
* Start over nearly from scratch whenever you need to:
    - Deploy in a standalone server
    - Visualize different aspects of your data
    - Scale up to larger (>100K) datasets

# Step-by-step data-science workflow

Here we'll show a simple, flexible, powerful, step-by-step workflow, explaining which open-source tools solve each of the problems involved:

-  Step 1: Get some data
-  Step 2: Prototype a plot in a notebook
-  Step 3: Define your domain model
-  Step 4: Get a widget-based UI for free
-  Step 5: Link your domain model to your visualization
-  Step 6: Widgets now control your interactive plots
-  Step 7: Deploy your dashboard

In [ ]:
import holoviews as hv
import geoviews as gv
import param, paramnb, parambokeh
import dask.dataframe as dd

from colorcet import cm
from holoviews.operation.datashader import datashade
from holoviews.operation import decimate
from holoviews.streams import RangeXY, PlotSize

## Step 1: Get some data

* Here we'll use a subset of the often-studied NYC Taxi dataset
* About 12 million points of GPS locations from taxis
* Stored in the efficient Parquet format for easy access
* Loaded into a Dask dataframe for multi-core<br>(and if needed, out-of-core or distributed) computation

In [ ]:
%time df = dd.read_parquet('../data/nyc_taxi_hours.parq/').persist()
print(len(df))
df.head(2)

## Step 2: Prototype a plot in a notebook

* A text-based representation isn't very useful for big datasets like this, so we need to build a plot
* But we don't want to start a software project, so we use HoloViews:
    - Simple, declarative way to annotate your data for visualization
    - Large library of Elements with associated visual representation
    - Elements combine (lay out or overlay) easily
* And we'll want live interactivity, so we'll use a Bokeh plotting extension
* Result:

In [ ]:
hv.extension('bokeh')

In [ ]:
points = hv.Points(df, kdims=['pickup_x', 'pickup_y'])
decimate(points)

Here ``Points`` declares an object wrapping `df` and visualized as a scatterplot, and `decimate` limits the number of points that will be sent to the browser to avoid crashing it.

As you can see, HoloViews makes it very simple to pop up a visualization of your data, getting *something* on screen with only a few characters of typing.  But it's not particularly pretty, so let's customize it a bit:

In [ ]:
options = dict(width=700, height=600, xaxis=None, yaxis=None, bgcolor='black')
points = points.opts(plot=options)
decimate(points)

That looks a bit better, but it's still decimating the data nearly beyond recognition, so let's try using Datashader to rasterize it into a fixed-size image to send to the browser:

In [ ]:
taxi_trips = datashade(points, x_sampling=1, y_sampling=1, cmap=cm['fire']).opts(plot=options)
taxi_trips

Ok, that looks good now; there's clearly lots to explore in this dataset.  To put it in context, let's overlay that on a map:

In [ ]:
taxi_trips = datashade(points, x_sampling=1, y_sampling=1, cmap=cm['fire']).opts(plot=options)
tiles = gv.WMTS('https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg')
tiles * taxi_trips

We could add lots more visual elements (laying out additional plots left and right, overlaying annotations, etc.), but let's say that this is our basic visualization we'll want to share. To sum up what we've done so far, here are the complete 10 lines of code required to generate this geo-located interactive plot of millions of datapoints in Jupyter:

```
import holoviews as hv, geoviews as gv, dask.dataframe as dd
from colorcet import cm; from holoviews.operation.datashader import datashade

hv.extension('bokeh')
df = dd.read_parquet('../data/nyc_taxi_hours.parq/').persist()
options = dict(width=700, height=600, xaxis=None, yaxis=None, bgcolor='black')
points = hv.Points(df, kdims=['pickup_x', 'pickup_y'])
taxi_trips = datashade(points, x_sampling=1, y_sampling=1, cmap=cm['fire']).opts(plot=options)
tiles = gv.WMTS('https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg')
tiles * taxi_trips
```

## Step 3: Define your domain model

Now that we've prototyped a nice plot, we could keep editing the code above to explore whatever aspects of the data we wished. However, often at this point we will want to start sharing our workflow with people who aren't familar with how to program visualizations in this way.   

So the next step: figure out what we want our intended user to be able to change, and declare those variables or parameters with:

  - type and range checking
  - documentation strings
  - default values
  
The Param library allows declaring Python attributes having these features (and more, such as dynamic values and inheritance), letting you set up a well-defined space for a user (or you!) to explore.

## NYC Taxi Parameters

In [ ]:
class NYCTaxiExplorer(hv.streams.Stream):
    alpha       = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    plot        = param.ObjectSelector(default="pickup", objects=["pickup","dropoff"])
    colormap    = param.ObjectSelector(default=cm["fire"], objects=cm.values())
    passengers  = param.Range(default=(0, 10), bounds=(0, 10), doc="""
        Filter for taxi trips by number of passengers""")

Each Parameter is a normal Python attribute, but with special checks and functions run automatically when getting or setting.

Parameters capture your goals and your knowledge about your domain, declaratively.

### Class level parameters

In [ ]:
NYCTaxiExplorer.alpha

In [ ]:
NYCTaxiExplorer.alpha = 0.5
NYCTaxiExplorer.alpha

### Validation

In [ ]:
try:
   NYCTaxiExplorer.alpha = '0'
except Exception as e:
    print(e) 

### Instance parameters

In [ ]:
explorer = NYCTaxiExplorer(alpha=0.6)
explorer.alpha

In [ ]:
NYCTaxiExplorer.alpha

## Step 4: Get a widget-based UI for free

* Parameters are purely declarative and independent of any widget toolkit, but contain all the information needed to build interactive widgets
* ParamNB generates UIs in Jupyter from Parameters, using ipywidgets

In [ ]:
paramnb.Widgets(NYCTaxiExplorer)

In [ ]:
NYCTaxiExplorer.passengers

* ipywidgets work with Jupyter Dashboards Server for deployment

* Declaration of parameters is independent of the UI library used
* ParamBokeh generates UIs from the same Parameters, using Bokeh widgets, either in Jupyter or in Bokeh Server

In [ ]:
parambokeh.Widgets(NYCTaxiExplorer)

## Step 5: Link your domain model to your visualization

We've now defined the space that's available for exploration, and the next step is to link up the parameter space with the code that specifies the plot:

In [ ]:
class NYCTaxiExplorer(hv.streams.Stream):
    alpha       = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    colormap    = param.ObjectSelector(default=cm["fire"], objects=cm.values())
    plot        = param.ObjectSelector(default="pickup",   objects=["pickup","dropoff"])
    passengers  = param.Range(default=(0, 10), bounds=(0, 10))

    def make_view(self, x_range=None, y_range=None, **kwargs):
        map_tiles = tiles.opts(style=dict(alpha=self.alpha), plot=options) 

        points = hv.Points(df, kdims=[self.plot+'_x', self.plot+'_y'], vdims=['passenger_count'])
        selected = points.select(passenger_count=self.passengers)
        taxi_trips = datashade(selected, x_sampling=1, y_sampling=1, cmap=self.colormap,
                               dynamic=False, x_range=x_range, y_range=y_range,
                               width=800, height=475)
        return map_tiles * taxi_trips

Note that the `NYCTaxiExplorer` class is entirely declarative (no widgets), and can be used "by hand" to provide range-checked and type-checked plotting for values from the declared parameter space:

In [ ]:
explorer = NYCTaxiExplorer(alpha=0.2, plot="dropoff")
explorer.make_view()

## Step 6: Widgets now control your interactive plots

But in practice, why not pop up the widgets to make it fully interactive?

In [ ]:
explorer = NYCTaxiExplorer()
paramnb.Widgets(explorer, callback=explorer.event)
hv.DynamicMap(explorer.make_view, streams=[explorer, RangeXY()])

In [ ]:
explorer = NYCTaxiExplorer()
parambokeh.Widgets(explorer, callback=explorer.event)
hv.DynamicMap(explorer.make_view, streams=[explorer, RangeXY()])

## Step 7: Deploy your dashboard

Ok, now you've got something worth sharing, running inside Jupyter.  But if you want to share your work with people who don't use Python, you'll now want to run a server with this same code.

* If you used **ParamBokeh**, deploy with **Bokeh Server**:
    - Write the above code to a file ``nyc_parambokeh.py``
    - Add ``, mode='server'`` to the ``Widgets()`` call to declare which object should be served
    - ``bokeh serve nyc_parambokeh.py``

* If you used **ParamNB**, deploy with **Jupyter Dashboard Server**:
    - Use [Jupyter Dashboards Extension](https://github.com/jupyter/dashboards) to select cells from the notebook to display
    - Use preview mode to see layout
    - Use [Jupyter Dashboards Server](https://github.com/jupyter-incubator/dashboards_server) to deploy
    - Note various caveats below

# Complete dashboard code

```
import holoviews as hv, geoviews as gv, param, parambokeh, dask.dataframe as dd

from colorcet import cm
from bokeh.models import WMTSTileSource
from holoviews.operation.datashader import datashade
from holoviews.streams import RangeXY, PlotSize

hv.extension('bokeh')

df = dd.read_parquet('./data/nyc_taxi.parq/').persist()
url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'
tiles = gv.WMTS(WMTSTileSource(url=url))
tile_options = dict(width=800,height=475,xaxis=None,yaxis=None,bgcolor='black',show_grid=False)

passenger_counts = (0, df.passenger_count.max().compute()+1)

class NYCTaxiExplorer(hv.streams.Stream):
    alpha      = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    colormap   = param.ObjectSelector(default=cm["fire"], objects=[cm[k] for k in cm.keys() if not '_' in k])
    plot       = param.ObjectSelector(default="pickup",   objects=["pickup","dropoff"])
    passengers = param.Range(default=passenger_counts, bounds=passenger_counts)
    output     = parambokeh.view.Plot()

    def make_view(self, x_range, y_range, alpha, colormap, plot, passengers, **kwargs):
        map_tiles = tiles(style=dict(alpha=alpha), plot=tile_options)
        points = hv.Points(df, kdims=[plot+'_x', plot+'_y'], vdims=['passenger_count'])
        if passengers != passenger_counts: points = points.select(passenger_count=passengers)
        taxi_trips = datashade(points, x_sampling=1, y_sampling=1, cmap=colormap,
                               dynamic=False, x_range=x_range, y_range=y_range)
        return map_tiles * taxi_trips

selector = NYCTaxiExplorer(name="NYC Taxi Trips")
selector.output = hv.DynamicMap(selector.make_view, streams=[selector, RangeXY(), PlotSize()])

parambokeh.Widgets(selector, view_position='right', callback=selector.event, mode='server')
```

# Branching out

The other sections in this tutorial will expand on steps in this workflow, providing more step-by-step instructions for each of the major tasks.  These techniques can create much more ambitious apps with very little additional code or effort:

* Adding additional linked or separate subplots of any type; see [2 - Annotating your data](./02-introduction-to-elements.ipynb) and [4 - Exploration with containers](./04-exploration-with-containers.ipynb).
* Declaring code that runs for clicking or selecting *within* the Bokeh plot; see [7 - Custom interactivity](./07-custom-interactivity.ipynb).
* Using multiple sets of widgets of many different types; see [ParamNB](https://github.com/ioam/paramnb) and [ParamBokeh](https://github.com/ioam/parambokeh).
* Using datasets too big for any one machine, with [Dask.Distributed](https://distributed.readthedocs.io).

# Future work

* Jupyter Dashboards Server not currently maintained; requires older ipywidgets version
* Bokeh Server is mature and well supported, but does not currently support drag-and-drop layout like Jupyter Dashboards does
* ParamBokeh and ParamNB still need some polishing and work to make them ready for widespread use
* E.g. ParamNB and ParamBokeh should provide more flexible widget layouts